In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, date
import csv
import time
import ssl
import os
import threading
ssl._create_default_https_context = ssl._create_unverified_context
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
DEBUG = False
time_data = [2024, 10, 28]
otc_list_path = r".\db\otc.csv"
twse_list_path = r".\db\twse.csv"
db_path = r"D:\Stock\db"

In [3]:
output_file_path = r"D:\Stock\daily_result\%s%s%s_trend.csv"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))
# output_file_path = r"D:\Stock\daily_result\%s%s%s_start.csv"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))

In [4]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

def print_log(ss, is_debug):
    if is_debug:
        print(ss)

In [5]:
def get_twse_stock_info(df, stock):
    target_data = df[df["股票代號"] == int(stock)]
    name = target_data.iloc[0]['股票名稱']
    priceEarningRatio = target_data.iloc[0]['本益比']
    yieldRatio = target_data.iloc[0]['殖利率(%)']
    priceBookRatio = target_data.iloc[0]['股價淨值比']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_otc_stock_info(df, stock):
    target_data = df[df['SecuritiesCompanyCode'] == stock]
    name = target_data.iloc[0]['CompanyName']
    priceEarningRatio = target_data.iloc[0]['PriceEarningRatio']
    dividendPerShare = target_data.iloc[0]['DividendPerShare']
    yieldRatio = target_data.iloc[0]['YieldRatio']
    priceBookRatio = target_data.iloc[0]['PriceBookRatio']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_twse_stock_db_info():
    link = 'http://www.twse.com.tw/exchangeReport/BWIBBU_ALL?response=open_data'
    df = pd.read_csv(link, encoding='utf_8_sig')
    return df

def get_otc_stock_db_info():
    link = 'http://www.tpex.org.tw/openapi/v1/tpex_mainboard_peratio_analysis'
    json_data = requests.get(link).json()
    df = pd.DataFrame.from_records(json_data)
    return df

In [6]:
def calculate_target_trend(vol_data, pri_data, pri_max_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -7
    bb = -28
    if len(vol_data[bb:aa]) == 0 or len(pri_data[bb:aa]) == 0:
        return False
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    smooth_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:]) > 450
        
        if not vol_valid:
            print_log("vol invalid", DEBUG)
            return False
        
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        # if pri_5ma[-1] == 0 or pri_10ma[-1] == 0 or pri_20ma[-1] == 0 or pri_60ma[-1] == 0:
        #     return False
        
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.03 and pri_cur / pri_5ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.983
        print_log("pri_cur / pri_5ma[-1]: %f"%(pri_cur / pri_5ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.03 and pri_cur / pri_10ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.989
        print_log("pri_cur / pri_10ma[-1]: %f"%(pri_cur / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.04 and pri_5ma[-1] / pri_10ma[-1] >= 0.975
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.974
        pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.98
        print_log("pri_5ma[-1] / pri_10ma[-1]: %f"%(pri_5ma[-1] / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.06 and pri_10ma[-1] / pri_20ma[-1] >= 0.98
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.045 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.09 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        print_log("pri_10ma[-1] / pri_20ma[-1]: %f"%(pri_10ma[-1] / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.07 and pri_cur / pri_20ma[-1] >= 0.99
        pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.12 and pri_cur / pri_20ma[-1] >= 0.995
        print_log("pri_cur / pri_20ma[-1]: %f"%(pri_cur / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.04
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.022
        pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.027
        print_log("pri_10ma[-1] / pri_60ma[-1]: %f"%(pri_10ma[-1] / pri_60ma[-1]), DEBUG and not pos_valid)
        
        if not pos_valid:
            print_log("pos invalid", DEBUG)
            return False
        
        trend_valid = trend_valid and pri_10ma[-1] / pri_10ma[-5] >= 0.985
        print_log("pri_10ma[-1] / pri_10ma[-5]: %f"%(pri_10ma[-1] / pri_10ma[-5]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.985
        trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.992
        print_log("pri_20ma[-1] / pri_20ma[-8]: %f"%(pri_20ma[-1] / pri_20ma[-8]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.035
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.023
        trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.021
        print_log("pri_60ma[-1] / pri_60ma[-10]: %f"%(pri_60ma[-1] / pri_60ma[-10]), DEBUG and not trend_valid)
        
        if not trend_valid:
            print_log("trend invalid", DEBUG)
            return False
        
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-2] > 0.955
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-8:].max() > 0.96
        smooth_valid = smooth_valid and pri_data[-1] / pri_max_data[-9:].max() > 0.94
        print_log("pri_data[-1] / pri_max_data[-10:]: %f"%(pri_data[-1] / pri_max_data[-8:].max()), DEBUG and not smooth_valid)
        
        if not smooth_valid:
            print_log("smooth invalid", DEBUG)
            return False
        
        # print(pri_cur / pri_5ma[-1], pri_cur / pri_10ma[-1], pri_5ma[-1] / pri_10ma[-1], pri_10ma[-1] / pri_20ma[-1], pri_cur / pri_20ma[-1] ,pri_10ma[-1] / pri_60ma[-1])
        # print(pri_20ma[-1] / pri_20ma[-8], pri_60ma[-1] / pri_60ma[-10])
    except:
        # print("?")
        return False
    # print(vol_valid , pos_valid , trend_valid)
    return vol_valid and pos_valid and trend_valid and smooth_valid

def calculate_target_bear_trend(vol_data, pri_data, pri_max_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -7
    bb = -28
    if len(vol_data[bb:aa]) == 0 or len(pri_data[bb:aa]) == 0:
        return False
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    smooth_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:]) > 450
        
        if not vol_valid:
            print_log("vol invalid", DEBUG)
            return False
        
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        # if pri_5ma[-1] == 0 or pri_10ma[-1] == 0 or pri_20ma[-1] == 0 or pri_60ma[-1] == 0:
        #     return False
        
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.03 and pri_cur / pri_5ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.017 and pri_cur / pri_5ma[-1] >= 0.96
        print_log("pri_cur / pri_5ma[-1]: %f"%(pri_cur / pri_5ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.03 and pri_cur / pri_10ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.011 and pri_cur / pri_10ma[-1] >= 0.96
        print_log("pri_cur / pri_10ma[-1]: %f"%(pri_cur / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.04 and pri_5ma[-1] / pri_10ma[-1] >= 0.975
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.974
        pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.02 and pri_5ma[-1] / pri_10ma[-1] >= 0.968
        print_log("pri_5ma[-1] / pri_10ma[-1]: %f"%(pri_5ma[-1] / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.06 and pri_10ma[-1] / pri_20ma[-1] >= 0.98
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.045 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.026 and pri_10ma[-1] / pri_20ma[-1] >= 0.91
        print_log("pri_10ma[-1] / pri_20ma[-1]: %f"%(pri_10ma[-1] / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.07 and pri_cur / pri_20ma[-1] >= 0.99
        pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.005 and pri_cur / pri_20ma[-1] >= 0.88
        print_log("pri_cur / pri_20ma[-1]: %f"%(pri_cur / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.04
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.022
        pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] <= 0.913
        print_log("pri_10ma[-1] / pri_60ma[-1]: %f"%(pri_10ma[-1] / pri_60ma[-1]), DEBUG and not pos_valid)
        
        if not pos_valid:
            print_log("pos invalid", DEBUG)
            return False
        
        trend_valid = trend_valid and pri_10ma[-1] / pri_10ma[-5] <= 1.015
        print_log("pri_10ma[-1] / pri_10ma[-5]: %f"%(pri_10ma[-1] / pri_10ma[-5]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.985
        trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] <= 1.008
        print_log("pri_20ma[-1] / pri_20ma[-8]: %f"%(pri_20ma[-1] / pri_20ma[-8]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.035
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.023
        trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] <= 0.919
        print_log("pri_60ma[-1] / pri_60ma[-10]: %f"%(pri_60ma[-1] / pri_60ma[-10]), DEBUG and not trend_valid)
        
        if not trend_valid:
            print_log("trend invalid", DEBUG)
            return False
        
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-2] < 1.045
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-8:].max() < 1.04
        smooth_valid = smooth_valid and pri_data[-1] / pri_max_data[-9:].max() < 1.06
        print_log("pri_data[-1] / pri_max_data[-10:]: %f"%(pri_data[-1] / pri_max_data[-8:].max()), DEBUG and not smooth_valid)
        
        if not smooth_valid:
            print_log("smooth invalid", DEBUG)
            return False
        
        # print(pri_cur / pri_5ma[-1], pri_cur / pri_10ma[-1], pri_5ma[-1] / pri_10ma[-1], pri_10ma[-1] / pri_20ma[-1], pri_cur / pri_20ma[-1] ,pri_10ma[-1] / pri_60ma[-1])
        # print(pri_20ma[-1] / pri_20ma[-8], pri_60ma[-1] / pri_60ma[-10])
    except:
        # print("?")
        return False
    # print(vol_valid , pos_valid , trend_valid)
    return vol_valid and pos_valid and trend_valid and smooth_valid

In [7]:
def analyze_stock(time_data, isOtc, target_list, thread_total = 1, thread_idx = 0):
    if isOtc:
        df = get_otc_stock_db_info()
        # data_path = r".\db\otc.csv"
        data_path = otc_list_path
    else:
        df = get_twse_stock_db_info()
        # data_path = r".\db\twse.csv"
        data_path = twse_list_path

    stock_list = []
    with open(data_path, newline='', encoding='utf_8_sig') as csvfile:
        line_list = csv.reader(csvfile)
        for line in line_list:
            stock_list.append(line[0])
    
    stock_list_len = len(stock_list)
    thread_item_num = np.ceil(stock_list_len / thread_total)
    idx_start = thread_item_num * thread_idx
    idx_end = idx_start + thread_item_num
    if thread_idx == thread_total - 1: # last thread
        idx_end = stock_list_len
    
    # target_list = []

    for stock in stock_list[int(idx_start):int(idx_end)]:
        # vol_data, pri_data, pri_max_data, df_all = get_time_duration_stock_info(time_data, stock, min_volumn=150, ma_num=1, isOtc=isOtc)
        stock_info_file_path = os.path.join(db_path, "%s.csv"%(stock))
        if not os.path.isfile(stock_info_file_path):
            continue
        df_all = pd.read_csv(stock_info_file_path)
        
        vol_data = df_all['成交張數'].values
        pri_data = df_all['收盤價'].values
        pri_max_data = df_all['最高價'].values
        pri_min_data = df_all['最低價'].values
        pri_sta_data = df_all['開盤價'].values
        
        if vol_data is None or pri_data is None:
            continue
        valid = calculate_target_trend(vol_data, pri_data, pri_max_data)
        # valid = calculate_target_bear_trend(vol_data, pri_data, pri_max_data)
        if valid:
            if isOtc:
                name, priceEarningRatio, yieldRatio, priceBookRatio = get_otc_stock_info(df, stock)
                target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 1])
                print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 1)
            else:
                name, priceEarningRatio, yieldRatio, priceBookRatio = get_twse_stock_info(df, stock)
                target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 0])
                print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 0)
        # break

In [8]:
target_list = []
df_all_ = analyze_stock(time_data, isOtc=True, target_list=target_list, thread_total = 1, thread_idx = 0)

1565 精華 18.91 3.28 1.76 211.5 1
4931 新盛力 27.42 2.32 2.09 36.3 1
6220 岳豐 N/A 2.95 1.29 27.0 1
8059 凱碩 N/A 0.00 3.59 36.2 1
8937 合騏 84.02 0.00 8.70 83.9 1


In [9]:
target_list = []
df_all_ = analyze_stock(time_data, isOtc=False, target_list=target_list, thread_total = 1, thread_idx = 0)

2357 華碩 13.42 2.92 1.65 595.0 0
2603 長榮 6.32 4.68 0.93 206.0 0
4571 鈞興-KY 22.31 1.9 3.16 169.0 0
4935 茂林-KY 19.97 1.98 1.17 75.2 0
6224 聚鼎 46.17 2.18 2.27 77.8 0
6415 矽力*-KY 152.12 0.39 5.87 528.0 0
9935 慶豐富 20.26 1.43 2.49 34.5 0
9938 百和 22.63 1.25 1.93 81.4 0


In [13]:
THREAD_NUM = 6
threads = []
target_list = []
for i in range(THREAD_NUM):
    threads.append(threading.Thread(target = analyze_stock, args = (time_data, False, target_list, THREAD_NUM, i)))
    threads[i].start()
    
for i in range(THREAD_NUM):
  threads[i].join()

print("Done.")

4571 鈞興-KY 22.31 1.9 3.16 169.0 0
4935 茂林-KY 19.97 1.98 1.17 75.2 0
2603 長榮 6.32 4.68 0.93 206.0 0
2357 華碩 13.42 2.92 1.65 595.0 0
6224 聚鼎 46.17 2.18 2.27 77.8 0
6415 矽力*-KY 152.12 0.39 5.87 528.0 0
9935 慶豐富 20.26 1.43 2.49 34.5 0
9938 百和 22.63 1.25 1.93 81.4 0
Done.


In [11]:
THREAD_NUM = 6
threads = []
target_list_otc = []
for i in range(THREAD_NUM):
    threads.append(threading.Thread(target = analyze_stock, args = (time_data, True, target_list_otc, THREAD_NUM, i)))
    threads[i].start()
    
for i in range(THREAD_NUM):
  threads[i].join()

print("Done.")

1565 精華 18.91 3.28 1.76 211.5 1
8059 凱碩 N/A 0.00 3.59 36.2 1
4931 新盛力 27.42 2.32 2.09 36.3 1
8937 合騏 84.02 0.00 8.70 83.9 1
6220 岳豐 N/A 2.95 1.29 27.0 1
Done.
